# Botlzmann Machines

BM is a energy-based model, over a binary vector $x \in \{ 0, 1 \} ^d$.
The joint probability distribution is adefined as:
$$P(x) = \frac{\exp(-E(x))}{Z}$$
$$Z = \sum_x \exp (-E(x))$$
$$E(x) = -x^TUx - b^tx$$
with $U \in \mathbb{R}^{d*d}$ the weight matrix, and $b \in \mathbb{R}^d$ the vector of bias.  
The probability one one unit being on is given by a linear model of the other units.  
When we add latent variables, we get a model with giher-order interractions between visible units. BM is a universal approximator of distributions over discrete variables.  

Diving between visible and hidden units, the model became:
$$E(v, h) = -v^TRv - v^TWh - h^TSh - b^Tv - c^Th$$

Boltzman machines are trained using maximum likelihood, but the partition function is intractable so MCMCM approximations are used.  
The learning rule is local: update a weight between 2 units only depends on the statistics of the 2 units (but the rest of the network shape those statistics).

# Restricted Botlzmann Machines

RMB is a bipartite undirected graphical model, with a layer of vivisble and a layer of latent variables, and no connection between units in the same layer.  
The energy function is:

$$E(v, h) = - b^Tv - c^Th - v^TWh$$

$Z$ is also intractable for RMBs.

## Conditional Distributions

The conditional distributions $P(h|v)$ and $P(v|h)$ are factorial and relatively easy to compute and sample from
$$P(h_j = 1 | v) = \sigma (c_j + v^T W_{:,j})$$
$$P(v_i = 1 | h) = \sigma (b_i + W_{i}h)$$
We get the factorial distributions:
$$P(h|v) = \prod_{j=1}^{n_j} \sigma \left( (2h - 1) \odot (c + W^Tv) \right) _j$$
$$P(v|h) = \prod_{i=1}^{n_v} \sigma \left( (2v - 1) \odot (b + Wh) \right) _i$$

## Training RBMs

Because of the easy sampling from $P(h|v)$ and $P(v|h)$, they can be trained with efficient block Gibbs sampling methods such as CD, PCD, ratio matching.

# Deep Belief Network

DBNs helped in 2006 to train the first deeps architecture, and had an important history role in the renaissance of deep learning, but they are rarelly used today.

A DBN has layers of binary latent variables, and a layer of visible variables (binary or real). There is no intralayer connection, and usually every unit is connected to every other unit in its neighboring layers.  
The connections between the top 2 latent layers are undirected, and all other connections are directed (toward visible units).  
A DBN with $l$ latent layers has $l$ weights matrices $W^{(i)}$ and $l+1$ bias vectors $b^{(i)}$:

$$P(h_i^{(k)} = 1 | h^{(k+1)}) = \sigma \left( b_i^{(k)} + W_{:,i}^{(k+1)T}h^{(k+1)} \right)$$
$$P(v_i = 1 | h^{(1)}) = \sigma \left( b_i^{(0)} + W_{:,i}^{(1)T}h^{(1)} \right)$$

If $v$ is real, we get instead:
$$v \sim \mathcal{N}(b^{(0)} + W^{(1)T}h^{(1)}, \beta^{-1})$$
with $\beta$ diagonal matrix.  
A DBM with only one hidden layer is an RBM.  
To draw samples, we use Gibbs like sampling in the top-two hidden layers like in RBMs, then we use ancestral sampling on the rest of the model.  

Inference is intractable due to the undirected connections and the explaining away effect of the directec connections. The ELBO is also intractable.  

To train a DBN, we start with an RBM and maximize:
$$\mathbb{E}_{v \sim p_\text{data}} \log p(v)$$
using CD algorithms. This correspond to the parameters of the first layer of the DBN.  

We train a second RBM to maximize:
$$\mathbb{E}_{v \sim p_\text{data}} \mathbb{E}_{h^{(1)} \sim p^{(1)}(h^{(1)}|v)} \log p^{(2)}(h^{(1)})$$

with $p^{(1)}$ the distribution of the already trained RBM, and $p^{(2)}$ the distribution of the RBM we are training. We get the parameters of the second layers of the RBM.  
We can repeat this several times to get as many layers as we want.  

The trained DBN can be used as a generative model, but mostly we use them to initialize the weights of an MLP with sigmoid activations. We can add other layers and/or fine-tune the weights, to improve classification models.  

The choice of MLP is arbitrary and seems to work well in practice. We can construct a variatonal lower bound on the expectation of the hidden units defined by the DBN, but there is no proof that it provides a tight bound.

# Deep Boltzmann Machines

A DBM has several layer of latent variables, each unit connected only with units in the neighboring layers.  
A DBM can be arranged into a bipartite graph, with odd layers on one side and even layers on another side. For Gibbs Sampling, we only need 2 blocks of update. We can sample silmultaneouly from odd layers, and then from even layers.

Let's take an example of a DBM with 3 hidden layers:
$$E(v, h^{(1)}, h^{(2)}, h^{(3)}) = -v^TW^{(1)}h^{(1)} - h^{(1)T}W^{(2)}h^{(2)} - h^{(2)T}W^{(3)}h^{(3)}$$
The activation probabilites are given by:
$$P(v_i = 1 | h^{(1)}) = \sigma (W^{(1)}_{i,:} h^{(1)})$$
$$P(h_i^{(1)} = 1 | v, h^{(2)}) = \sigma (v^TW_{:,i}^{(1)} + W^{(2)}_{i,:} h^{(2)})$$
$$P(h_k^{(2)} = 1 | h^{(1)}) = \sigma (h^{(1)T}W^{(1)}_{:,k})$$

## Interesting Properties


Comapred to DBNs, DBMs have a simpler posterior $P(h|v)$, but we get richer approximations.  
Using DBNs, we get a lower bound for the mean-field approximation of the hidden units of a MLP, but this bound is not optimized in any way, and it may be far from optimial.  
With DBMs, we can optimize the variational lower bound.  

Sampling from a DBM is generally harder than for DBNs, because we need to use MCMC across all layers.

## DBM Mean Field Inference

We can approximate the posterior $p(h|v)$ with a varitational approximation, using the mean field approach.  
We take an example with 2 hidden layers. Let $Q(h^{(1)}, h^{(2)}|v)$ the approximation of $P(h^{(1)}, h^{(2)}|v)$. Th mean field assumption implies:
$$Q(h^{(1)}, h^{(2)}|v) = \prod_j Q(h_j^{(1)}|v) \prod_k Q(h_k^{(2)}|v)$$

The inference process must be run every time we use a new $v$.  
To measure how well $Q$ fits $P$ we use $\text{KL}(Q||P)$.  

We parametrize $Q$ as a product of Bernouilli distributions, with 2 vectors::
$$Q(h_j^{(1)} = 1 | v) = \hat{h}_j^{(1)}$$
$$Q(h_k^{(2)} = 1 | v) = \hat{h}_k^{(2)}$$
$Q$ can be written as:
$$Q(h^{(1)}, h^{(2)}|v) = \prod_j (\hat{h}_j^{(1)} )^{h_j^{(1)}} (1 - \hat{h}_j^{(1)}) ^{1 - h_j^{(1)}} \prod_k (\hat{h}_k^{(2)}) ^{h_k^{(2)}}  (1 - \hat{h}_k^{(2)}) ^{1 - h_k^{(2)}}$$

We can solve using the mean field equations:
$$\tilde{q}(h_i|v) = \exp ( \mathbb{E}_{h_{-i} \sim q(h_{-i}|v) } \log \tilde{p}(v,h) )$$

We get the following updates:
$$\hat{h}_j^{(1)} = \sigma \left( \sum_i v_i W_{ij}^{(1)} + \sum_{k'} W_{jk'}^{(2)} \hat{h}_{k'}^{(2)} \right)$$
$$\hat{h}_k^{(2)} = \sigma \left( \sum_{j'} W_{j'k}^{(2)} \hat{h}_{j'}^{(1)} \right)$$

We have a local maximum at a fixed point of these equations. We iterativerely update the parameters one by one untl convergence.

## DBM Parameter Learning

Variational inference build a distribution $Q(h|v)$ that approximates $P(h|v)$.  
We also need to maximize a ELBO $\mathcal{L}(v, Q, \theta)$, variational lower bound on $\log P(v;\theta)$.  
DBMs are usually trained with variational stochastic maximum likelihood.

## Layer-Wise Pretraining

Training a DBM using SML may fail to learn to reprent the distribution, or as well as an RBM.  

A solution is greedy layer-wise pretraining. We train each layer separately as an RBM. The first layer is trained to the model the input data, and other layers to model samples from the previous layers posterior distribution.  
Then the final DBM is trained with PCD, which will do a small change in the parameters.

## Jointly training DBM

DBM for classification use greedy unsupervised pretraining, and use an MLP on top of the extracted features. But it's hard to evaluate performance of the first trained RBMs. And the MLP loose a lot of advantages of DBM.  

One solution is centered DBM: we reparametrize the model to be able to train it without pretraining. It produces high-quality samples and good classification, but not as good as MLP classifiers.  

The energy function of a RBM is:
$$E(x) = - x^TUx - b^Tx$$
The centendered version introduces an hyperparemeter, the vector $\mu$:
$$E'(x) = - (x - \mu)U (x - \mu) - (x - \mu)^Tb$$
It is chosen so that $x - \mu \approx 0$. This doesn't change the result, but it yields a hessian matrix for the gradient of the likelihood to be better conditionned.  

Another solution is multi-prediction DBM, it doesn't have good likelihood or samples, but great classification results and can reason about missing inputs.

# Boltzmann Machines for Real-Valued Data

Botlzmann Machine are manipulating binary data. We can use real values $[0,1]$ by using the propability of being $1$, but this is not really satisfying, and lead to poor samples.

## Gaussian-Bernoulli RBMs

They have binary hidden units and Gaussian visible units, whose mean is a function of the hidden units. It is parametrized using a a precision or a covariance matrix:
$$p(v|h) = \mathcal{N}(Wh, \beta^{-1})$$

One way to define the energy function is:
$$E(v, h) = \frac{1}{2}v^T (\beta \odot v) - (v \odot \beta)^TWh - b^Th  $$

## Undirected Models of Conditional Covariance

693